In [9]:
import triton
import triton.language as tl
import torch

In [10]:
def _out_hw(H, W, KH, KW, SH, SW, PH, PW, DH, DW):
    H_OUT = (H + 2*PH - DH*(KH-1) - 1)//SH + 1
    W_OUT = (W + 2*PW - DW*(KW-1) - 1)//SW + 1
    return H_OUT, W_OUT

# col2img

In [14]:
import torch, torch.nn.functional as F

device = "cuda"
dtype  = torch.float16  # можно torch.float32 с TF32
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
try:
    torch.set_float32_matmul_precision("high")
except Exception:
    pass

# размеры
N, C, H, W   = 2, 3, 512, 640
OUT, KH, KW  = 16, 3, 3
SH, SW       = 2, 2
PH, PW       = 1, 1
DH, DW       = 1, 1

x = torch.randn(N, C, H, W, device=device, dtype=dtype)
w = torch.randn(OUT, C, KH, KW, device=device, dtype=dtype)
b = torch.randn(OUT, device=device, dtype=dtype)

H_OUT = (H + 2*PH - DH*(KH-1) - 1)//SH + 1
W_OUT = (W + 2*PW - DW*(KW-1) - 1)//SW + 1
L     = H_OUT * W_OUT
K     = C * KH * KW

# --- эталон conv2d ---
for _ in range(5):
    y_ref = F.conv2d(x, w, b, stride=(SH,SW), padding=(PH,PW), dilation=(DH,DW))
torch.cuda.synchronize()
s0,e0 = torch.cuda.Event(True), torch.cuda.Event(True)
s0.record()
for _ in range(20):
    y_ref = F.conv2d(x, w, b, stride=(SH,SW), padding=(PH,PW), dilation=(DH,DW))
e0.record(); torch.cuda.synchronize()
t_ref_ms = s0.elapsed_time(e0)/20.0

# --- im2col + ОДИН большой GEMM + fold ---
for _ in range(5):
    X_col = F.unfold(x, (KH,KW), dilation=(DH,DW), padding=(PH,PW), stride=(SH,SW))  # [N, K, L]
    # A2d: (N*L, K)
    A2d = X_col.transpose(1, 2).reshape(-1, K).contiguous()
    # B2d: (K, OUT)
    B2d = w.view(OUT, -1).transpose(0, 1).contiguous()
    Y2d = A2d @ B2d                                       # (N*L, OUT) — ОДИН GEMM!
    Y2d = Y2d + b.view(1, OUT)
    Y_col = Y2d.view(N, L, OUT).transpose(1, 2).contiguous()
    y_i2c = F.fold(Y_col, (H_OUT, W_OUT), kernel_size=1)

torch.cuda.synchronize()
s1,e1 = torch.cuda.Event(True), torch.cuda.Event(True)
s1.record()
for _ in range(20):
    X_col = F.unfold(x, (KH,KW), dilation=(DH,DW), padding=(PH,PW), stride=(SH,SW))  # [N,K,L]
    A2d = X_col.transpose(1, 2).reshape(-1, K).contiguous()    # (N*L, K)
    B2d = w.view(OUT, -1).transpose(0, 1).contiguous()         # (K, OUT)
    Y2d = A2d @ B2d
    Y2d = Y2d + b.view(1, OUT)
    Y_col = Y2d.view(N, L, OUT).transpose(1, 2).contiguous()   # [N, OUT, L]
    y_i2c = F.fold(Y_col, (H_OUT, W_OUT), kernel_size=1)
e1.record(); torch.cuda.synchronize()
t_i2c_ms = s1.elapsed_time(e1)/20.0

# проверка и метрики
torch.testing.assert_close(y_i2c, y_ref, rtol=1e-2, atol=1e-2)
max_abs = (y_i2c - y_ref).abs().max().item()
flops = 2.0 * N * OUT * H_OUT * W_OUT * K
gflops_ref = flops/(t_ref_ms*1e6)
gflops_i2c = flops/(t_i2c_ms*1e6)

print(f"F.conv2d    : {t_ref_ms:.3f} ms | ~{gflops_ref:,.1f} GFLOP/s")
print(f"im2col+GEMM : {t_i2c_ms:.3f} ms | ~{gflops_i2c:,.1f} GFLOP/s")
print(f"Speedup (i2c vs conv2d): {t_ref_ms/t_i2c_ms:.3f}×")
print(f"Max |diff|: {max_abs:.3e} ({dtype=})")


F.conv2d    : 0.087 ms | ~1,635.0 GFLOP/s
im2col+GEMM : 0.537 ms | ~263.8 GFLOP/s
Speedup (i2c vs conv2d): 0.161×
Max |diff|: 1.562e-02 (dtype=torch.float16)


In [17]:
# %% Sparse-by-K (structured) for Conv2d -> im2col -> GEMM
import torch, torch.nn.functional as F
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
try: torch.set_float32_matmul_precision("high")
except: pass

device = "cuda"
dtype  = torch.float16  # оставь fp16 для скорости (можно fp32)

# ----- ПАРАМЕТРЫ -----
N, C, H, W   = 4, 64, 128, 128     # batch, in_channels, H, W
OUT, KH, KW  = 128, 3, 3           # out_channels, kernel
SH, SW       = 1, 1
PH, PW       = 1, 1
DH, DW       = 1, 1

# целевая структурная разреженность по K (0.0..0.9). Пример: 0.5 = выкинем 50% K-строк
target_sparsity = 0.5

# ----- ДАННЫЕ -----
g = torch.Generator(device=device).manual_seed(0)
x = torch.randn(N, C, H, W, device=device, dtype=dtype, generator=g)
w = torch.randn(OUT, C, KH, KW, device=device, dtype=dtype, generator=g)
b = torch.randn(OUT, device=device, dtype=dtype, generator=g)

# ----- ВЫХОДНЫЕ РАЗМЕРЫ -----
H_OUT = (H + 2*PH - DH*(KH-1) - 1)//SH + 1
W_OUT = (W + 2*PW - DW*(KW-1) - 1)//SW + 1
L     = H_OUT * W_OUT
K     = C * KH * KW

print(f"Input:  {tuple(x.shape)}, Weights: {tuple(w.shape)}, Bias: {tuple(b.shape)}")
print(f"Output: (N, OUT, H_OUT, W_OUT) = ({N}, {OUT}, {H_OUT}, {W_OUT})")
print(f"K = C*KH*KW = {K}, L = H_OUT*W_OUT = {L}")

# ===================== БАЗА: cuDNN Conv2d =====================
for _ in range(5):
    y_ref = F.conv2d(x, w, b, stride=(SH,SW), padding=(PH,PW), dilation=(DH,DW))

torch.cuda.synchronize()
e0s, e0e = torch.cuda.Event(True), torch.cuda.Event(True)
e0s.record()
for _ in range(20):
    y_ref = F.conv2d(x, w, b, stride=(SH,SW), padding=(PH,PW), dilation=(DH,DW))
e0e.record(); torch.cuda.synchronize()
t_conv_ms = e0s.elapsed_time(e0e) / 20.0

# ===================== БАЗА: im2col + 1 GEMM =====================
# 1) im2col
for _ in range(3):
    X_col = F.unfold(x, (KH,KW), dilation=(DH,DW), padding=(PH,PW), stride=(SH,SW))  # [N, K, L]

torch.cuda.synchronize()
e1s, e1e = torch.cuda.Event(True), torch.cuda.Event(True)
e1s.record()
for _ in range(20):
    X_col = F.unfold(x, (KH,KW), dilation=(DH,DW), padding=(PH,PW), stride=(SH,SW))
e1e.record(); torch.cuda.synchronize()
t_unfold_ms = e1s.elapsed_time(e1e) / 20.0

# 2) A = (N*L, K)
A2d = X_col.transpose(1, 2).reshape(-1, K).contiguous()
# 3) B = (K, OUT)
B2d = w.view(OUT, -1).transpose(0, 1).contiguous()

# прогрев GEMM
for _ in range(3):
    Y2d = A2d @ B2d

torch.cuda.synchronize()
e2s, e2e = torch.cuda.Event(True), torch.cuda.Event(True)
e2s.record()
for _ in range(50):
    Y2d = A2d @ B2d
e2e.record(); torch.cuda.synchronize()
t_gemm_dense_ms = e2s.elapsed_time(e2e) / 50.0

# 4) bias + fold
Y2d_bias = Y2d + b.view(1, OUT)
Y_col    = Y2d_bias.view(N, L, OUT).transpose(1, 2).contiguous()
y_i2c    = F.fold(Y_col, (H_OUT, W_OUT), kernel_size=1)

# сверка корректности с conv2d (без спарсификации — должны совпасть плотно)
try:
    torch.testing.assert_close(y_i2c, y_ref, rtol=1e-2, atol=1e-2)
    print("✅ Dense im2col+GEMM совпал с F.conv2d (fp16, rtol=1e-2, atol=1e-2).")
except AssertionError as e:
    print("⚠️ Dense путь отличается от F.conv2d больше допуска:", str(e).splitlines()[0])

# ===================== СПАРСИФИКАЦИЯ ПО K (СТРУКТУРНАЯ) =====================
# Идея: B2d имеет форму (K, OUT). Обнулим "неважные" строки (общий вклад по OUT мал),
# затем УДАЛИМ эти строки из B2d и соответствующие столбцы из A2d → получим меньший GEMM: (N*L, K_keep) @ (K_keep, OUT)

# 1) оценка важности строк B2d: возьмем L2-норму по оси OUT
row_importance = B2d.float().pow(2).sum(dim=1)  # [K], считаем в fp32 ради стабильности
# 2) отберём верхние (1 - sparsity) долю строк
k_keep = max(1, int((1.0 - target_sparsity) * K))
keep_idx = torch.topk(row_importance, k_keep, largest=True, sorted=False).indices
keep_idx, _ = torch.sort(keep_idx)  # стабильный порядок
k_pruned = K - k_keep

print(f"\nStructured K-pruning: target_sparsity={target_sparsity:.2f} -> keep {k_keep}/{K} (−{k_pruned})")

# 3) сжимаем A и B
A2d_sp = A2d.index_select(1, keep_idx)      # (N*L, K_keep)
B2d_sp = B2d.index_select(0, keep_idx)      # (K_keep, OUT)

# 4) GEMM на суженных матрицах
for _ in range(3):
    Y2d_sp = A2d_sp @ B2d_sp

torch.cuda.synchronize()
e3s, e3e = torch.cuda.Event(True), torch.cuda.Event(True)
e3s.record()
for _ in range(50):
    Y2d_sp = A2d_sp @ B2d_sp
e3e.record(); torch.cuda.synchronize()
t_gemm_sparse_ms = e3s.elapsed_time(e3e) / 50.0

# 5) bias + сборка обратно
Y2d_sp = Y2d_sp + b.view(1, OUT)
Y_col_sp = Y2d_sp.view(N, L, OUT).transpose(1, 2).contiguous()
y_i2c_sp = F.fold(Y_col_sp, (H_OUT, W_OUT), kernel_size=1)

# ОЦЕНКА РАЗЛИЧИЙ (после прунинга ожидается отличаться!)
diff_max = (y_i2c_sp - y_ref).abs().max().item()
mse      = torch.mean((y_i2c_sp - y_ref)**2).item()

# ===================== СВОДКА ВРЕМЕНИ И GFLOP/s =====================
# Теоретические FLOPs одного GEMM: 2 * (N*L) * K_eff * OUT
def gflops(nl, kk, oo, ms):
    return (2.0 * nl * kk * oo) / (ms * 1e6)

NL = N * L
gflops_dense = gflops(NL, K,      OUT, t_gemm_dense_ms)
gflops_sp    = gflops(NL, k_keep, OUT, t_gemm_sparse_ms)

print("\n=== Timing (ms) ===")
print(f"F.conv2d (cuDNN):         {t_conv_ms:.3f} ms")
print(f"unfold (im2col):          {t_unfold_ms:.3f} ms")
print(f"GEMM dense (N*L,K)x(K,O): {t_gemm_dense_ms:.3f} ms  | ~{gflops_dense:,.1f} GFLOP/s")
print(f"GEMM pruned (K_keep):     {t_gemm_sparse_ms:.3f} ms  | ~{gflops_sp:,.1f} GFLOP/s")
print(f"Speedup GEMM (pruned vs dense): {t_gemm_dense_ms / t_gemm_sparse_ms:.2f}×")

print("\n=== Quality vs F.conv2d ===")
print(f"Max |diff| after K-prune:  {diff_max:.3e} (dtype={dtype})")
print(f"MSE after K-prune:         {mse:.3e}")

# Пояснение:
# - ускорение приходит за счёт уменьшения K (структурный prune), поэтому GEMM становится меньше.
# - это *реальный* ускор (без спец. sparse-ядра), работает на стандартном torch.matmul.
# - качество будет зависеть от заданной target_sparsity. Подбери порог/маску из обучения (L1/L2/OBS и т.д.).


Input:  (4, 64, 128, 128), Weights: (128, 64, 3, 3), Bias: (128,)
Output: (N, OUT, H_OUT, W_OUT) = (4, 128, 128, 128)
K = C*KH*KW = 576, L = H_OUT*W_OUT = 16384
✅ Dense im2col+GEMM совпал с F.conv2d (fp16, rtol=1e-2, atol=1e-2).

Structured K-pruning: target_sparsity=0.50 -> keep 288/576 (−288)

=== Timing (ms) ===
F.conv2d (cuDNN):         0.478 ms
unfold (im2col):          0.379 ms
GEMM dense (N*L,K)x(K,O): 0.410 ms  | ~23,572.9 GFLOP/s
GEMM pruned (K_keep):     0.134 ms  | ~35,932.0 GFLOP/s
Speedup GEMM (pruned vs dense): 3.05×

=== Quality vs F.conv2d ===
Max |diff| after K-prune:  9.162e+01 (dtype=torch.float16)
MSE after K-prune:         2.580e+02


In [18]:
# implicit_conv_triton.py
import torch
import triton
import triton.language as tl
import math

# ---------- Triton kernel ----------
# Простая версия: один блок обрабатывает несколько выходных точек по пространству и по выходным каналам.
# Ограничение: kernel заточен под kernel_size=3, stride=1, padding=1 (можно расширить).
@triton.jit
def conv3x3_fused_kernel(
    # pointers
    x_ptr, w_ptr, b_ptr, y_ptr,
    # sizes
    N, C, H, W, OUT, H_OUT, W_OUT,
    # strides (in elements, не байты)
    stride_xn, stride_xc, stride_xh, stride_xw,
    stride_wn, stride_wc, stride_wh, stride_ww,
    stride_yn, stride_yc, stride_yh, stride_yw,
    # tile sizes
    OUT_BLOCK, OUT_PIXELS, C_block
):
    pid = tl.program_id(0)   # индекс блока по output (блок покрывает OUT_BLOCK выходных каналов и OUT_PIXELS позиций)
    # block mapping: каждое pid обрабатывает (out_c_base, pixel_base)
    out_c_base = (pid // ((H_OUT*W_OUT + OUT_PIXELS - 1) // OUT_PIXELS)) * OUT_BLOCK
    pix_block_id = pid % ((H_OUT*W_OUT + OUT_PIXELS - 1) // OUT_PIXELS)
    pix_base = pix_block_id * OUT_PIXELS

    # массив выходных каналов и позиций в блоке
    out_cs = out_c_base + tl.arange(0, OUT_BLOCK)
    pixs = pix_base + tl.arange(0, OUT_PIXELS)

    # маски
    mask_outc = out_cs < OUT
    mask_pix = pixs < (H_OUT*W_OUT)

    # координаты пикселей (h,w) из индекса pix (0..L-1)
    h_idx = (pixs // W_OUT)
    w_idx = (pixs % W_OUT)

    # адреса для output: y[n, out_c, h, w]
    # Мы не параллелим по N здесь - можно добавить внешний цикл по n
    n = 0  # prototype for batch=1 only; расширяй по необходимости

    # инициализация аккумуляторов
    acc = tl.zeros((OUT_BLOCK, OUT_PIXELS), dtype=tl.float32)

    # проходим по input channels блоками C_block
    for c_off in range(0, C, C_block):
        # загрузим C_block входных каналов для всех пикселей
        cc = c_off + tl.arange(0, C_block)
        mask_c = cc < C

        # для каждого kk в C_block и для каждой позиции читаем 3x3 патч
        # читаем элементы с учётом padding=1, stride=1, dilation=1
        for kh in range(-1, 2):
            for kw in range(-1, 2):
                # входные координаты
                ih = h_idx + kh
                iw = w_idx + kw

                # mask по границам
                valid_hw = (ih >= 0) & (ih < H) & (iw >= 0) & (iw < W)

                # строим адреса x[n, cc, ih, iw] (векторизованные)
                # адрес = base + n*stride_xn + cc*stride_xc + ih*stride_xh + iw*stride_xw
                x_off = n * stride_xn + cc[:, None] * stride_xc + ih[None, :] * stride_xh + iw[None, :] * stride_xw
                # приводим к указателю
                x_vals = tl.load(x_ptr + x_off, mask=(mask_c[:, None] & valid_hw[None, :]), other=0.0).to(tl.float32)  # shape (C_block, OUT_PIXELS)

                # веса: w[out_c, cc, kh+1, kw+1]
                kc = cc[:, None]  # shape (C_block, 1)
                oc = out_cs[:, None]  # shape (OUT_BLOCK, 1)
                w_off = oc[:, None, None] * stride_wn + kc[None, :, None] * stride_wc + (kh+1) * stride_wh + (kw+1) * stride_ww
                w_vals = tl.load(w_ptr + w_off, mask=(mask_outc[:, None, None] & mask_c[None, :, None]), other=0.0).to(tl.float32)  # (OUT_BLOCK, C_block, 1)

                # умножение и накопление: acc[out_c, pix] += sum_c ( w[out_c,c] * x[c,pix] )
                # преобразуем: w_vals (OUT_BLOCK, C_block) , x_vals (C_block, OUT_PIXELS)
                # Сделаем мат-муль частично
                # Приведём w_vals к (OUT_BLOCK, C_block) и умножим на x_vals (C_block, OUT_PIXELS)
                # Сделаем явный вклад:
                # Note: это не оптимально, но корректно для прототипа
                for oc_i in range(OUT_BLOCK):
                    w_row = w_vals[oc_i, :, 0]  # (C_block,)
                    # умножение по C_block: (C_block,) @ (C_block, OUT_PIXELS) -> (OUT_PIXELS,)
                    dot = tl.dot(w_row, x_vals)
                    acc = tl.where(mask_outc[oc_i], acc, acc)  # no-op to keep shape; accumulation below
                    # аккумулируем в соответствующую строку
                    acc = acc + tl.where(mask_outc[oc_i, None], tl.reshape(dot, (1, OUT_PIXELS)).to(tl.float32), tl.zeros((OUT_BLOCK, OUT_PIXELS), dtype=tl.float32))

    # После накопления добавляем bias и записываем
    # bias: b[out_c]
    b_vals = tl.load(b_ptr + out_cs, mask=mask_outc, other=0.0).to(tl.float32)  # (OUT_BLOCK,)
    # добавим bias и записываем y[n, out_c, h, w]
    for i_oc in range(OUT_BLOCK):
        if not mask_outc[i_oc]:
            continue
        y_vals = acc[i_oc] + b_vals[i_oc]
        # адреса для записи y
        y_off = n * stride_yn + out_cs[i_oc] * stride_yc + h_idx * stride_yh + w_idx * stride_yw
        tl.store(y_ptr + y_off, y_vals.to(tl.float32), mask=mask_pix)

# ---------- Python harness ----------
def run_triton_conv3x3(x, w, b):
    # x: (N,C,H,W), w: (OUT,C,3,3), b:(OUT,)
    assert x.is_cuda and w.is_cuda and b.is_cuda
    N, C, H, W = x.shape
    OUT, Cw, KH, KW = w.shape
    assert KH == 3 and KW == 3
    H_OUT = H
    W_OUT = W

    # strides (in elements)
    # Compute contiguous strides
    stride_xn = x.stride(0)
    stride_xc = x.stride(1)
    stride_xh = x.stride(2)
    stride_xw = x.stride(3)
    stride_wn = w.stride(0)
    stride_wc = w.stride(1)
    stride_wh = w.stride(2)
    stride_ww = w.stride(3)
    y = torch.zeros((N, OUT, H_OUT, W_OUT), device=x.device, dtype=x.dtype)
    stride_yn = y.stride(0)
    stride_yc = y.stride(1)
    stride_yh = y.stride(2)
    stride_yw = y.stride(3)

    # параметры блокировки (настрой)
    OUT_BLOCK = 8      # сколько выходных каналов в блоке
    OUT_PIXELS = 16    # сколько пространственных позиций в блоке
    C_block = 16       # блок по входным каналам

    # grid size: количество блоков = ceil(OUT/OUT_BLOCK) * ceil(L/OUT_PIXELS)
    grid_x = (math.ceil(OUT / OUT_BLOCK) * math.ceil((H_OUT * W_OUT) / OUT_PIXELS),)

    # вызов kernel (для простоты: batch size = 1 поддержан в ядре; для N>1 - можно обойти циклом по n)
    conv3x3_fused_kernel[grid_x](
        x, w, b, y,
        N, C, H, W, OUT, H_OUT, W_OUT,
        stride_xn, stride_xc, stride_xh, stride_xw,
        stride_wn, stride_wc, stride_wh, stride_ww,
        stride_yn, stride_yc, stride_yh, stride_yw,
        OUT_BLOCK, OUT_PIXELS, C_block
    )
    return y


In [20]:
import torch

# подготовим данные (пример для N=1)
x_small = x[:1].contiguous()          # (1,C,H,W)
w_small = w.contiguous()               # (OUT,C,3,3)
b_small = b.contiguous()
y_tr = run_triton_conv3x3(x_small, w_small, b_small)
# сравним с F.conv2d
y_ref = F.conv2d(x_small, w_small, b_small, padding=1)
torch.testing.assert_close(y_tr, y_ref, rtol=1e-2, atol=1e-2)

CompilationError: at 20:26:
    stride_yn, stride_yc, stride_yh, stride_yw,
    # tile sizes
    OUT_BLOCK, OUT_PIXELS, C_block
):
    pid = tl.program_id(0)   # индекс блока по output (блок покрывает OUT_BLOCK выходных каналов и OUT_PIXELS позиций)
    # block mapping: каждое pid обрабатывает (out_c_base, pixel_base)
    out_c_base = (pid // ((H_OUT*W_OUT + OUT_PIXELS - 1) // OUT_PIXELS)) * OUT_BLOCK
    pix_block_id = pid % ((H_OUT*W_OUT + OUT_PIXELS - 1) // OUT_PIXELS)
    pix_base = pix_block_id * OUT_PIXELS

    # массив выходных каналов и позиций в блоке
    out_cs = out_c_base + tl.arange(0, OUT_BLOCK)
                          ^